In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('adult_one_hot.csv')
X_np = df.iloc[:,:-1].as_matrix()
X_np = (X_np - np.mean(X_np,axis=0)) / np.std(X_np,axis=0)
Y_np = np.expand_dims(df.iloc[:,-1].as_matrix(),1).astype(int)

## Logistic Regression

In [3]:
num_features = df.shape[1]-1
learning_rate = 0.1
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.zeros([num_features, 1])) # the variables will be all zeros initially
b = tf.Variable(tf.zeros([1]))

In [4]:
Yhat = tf.sigmoid(X @ W + b)
loss               = tf.reduce_mean(-(Y*tf.log(Yhat) + (1-Y)*tf.log(1-Yhat)))
# correct_prediction = tf.equal(tf.argmax(Yhat, 1), tf.argmax(Y, 1))           
# accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
optimizer = tf.train.GradientDescentOptimizer(learning_rate) 
update = optimizer.minimize(loss) 

In [5]:
num_epochs  = 500

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        _, loss_val = sess.run([update, loss], feed_dict={X: X_np, 
                                                      Y: Y_np})
        print("Epoch:",epoch,"Cost:",loss_val)        
#     print("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0 Cost: 0.69319
Epoch: 1 Cost: 0.662942
Epoch: 2 Cost: 0.637605
Epoch: 3 Cost: 0.616159
Epoch: 4 Cost: 0.59782
Epoch: 5 Cost: 0.581975
Epoch: 6 Cost: 0.568137
Epoch: 7 Cost: 0.555934
Epoch: 8 Cost: 0.545068
Epoch: 9 Cost: 0.535311
Epoch: 10 Cost: 0.526481
Epoch: 11 Cost: 0.518433
Epoch: 12 Cost: 0.511052
Epoch: 13 Cost: 0.504244
Epoch: 14 Cost: 0.497932
Epoch: 15 Cost: 0.492055
Epoch: 16 Cost: 0.486561
Epoch: 17 Cost: 0.481407
Epoch: 18 Cost: 0.476556
Epoch: 19 Cost: 0.471978
Epoch: 20 Cost: 0.467646
Epoch: 21 Cost: 0.46354
Epoch: 22 Cost: 0.459638
Epoch: 23 Cost: 0.455925
Epoch: 24 Cost: 0.452385
Epoch: 25 Cost: 0.449005
Epoch: 26 Cost: 0.445773
Epoch: 27 Cost: 0.442681
Epoch: 28 Cost: 0.439717
Epoch: 29 Cost: 0.436875
Epoch: 30 Cost: 0.434145
Epoch: 31 Cost: 0.431522
Epoch: 32 Cost: 0.428999
Epoch: 33 Cost: 0.426571
Epoch: 34 Cost: 0.424233
Epoch: 35 Cost: 0.421979
Epoch: 36 Cost: 0.419805
Epoch: 37 Cost: 0.417708
Epoch: 38 Cost: 0.415682
Epoch: 39 Cost: 0.413725
Epoch: 40 Cos

Epoch: 324 Cost: 0.328159
Epoch: 325 Cost: 0.328117
Epoch: 326 Cost: 0.328077
Epoch: 327 Cost: 0.328036
Epoch: 328 Cost: 0.327996
Epoch: 329 Cost: 0.327956
Epoch: 330 Cost: 0.327917
Epoch: 331 Cost: 0.327877
Epoch: 332 Cost: 0.327838
Epoch: 333 Cost: 0.3278
Epoch: 334 Cost: 0.327761
Epoch: 335 Cost: 0.327722
Epoch: 336 Cost: 0.327684
Epoch: 337 Cost: 0.327646
Epoch: 338 Cost: 0.327608
Epoch: 339 Cost: 0.32757
Epoch: 340 Cost: 0.327533
Epoch: 341 Cost: 0.327496
Epoch: 342 Cost: 0.327459
Epoch: 343 Cost: 0.327422
Epoch: 344 Cost: 0.327386
Epoch: 345 Cost: 0.32735
Epoch: 346 Cost: 0.327313
Epoch: 347 Cost: 0.327278
Epoch: 348 Cost: 0.327242
Epoch: 349 Cost: 0.327206
Epoch: 350 Cost: 0.327171
Epoch: 351 Cost: 0.327136
Epoch: 352 Cost: 0.327101
Epoch: 353 Cost: 0.327066
Epoch: 354 Cost: 0.327032
Epoch: 355 Cost: 0.326997
Epoch: 356 Cost: 0.326964
Epoch: 357 Cost: 0.326929
Epoch: 358 Cost: 0.326895
Epoch: 359 Cost: 0.326862
Epoch: 360 Cost: 0.326829
Epoch: 361 Cost: 0.326796
Epoch: 362 Cost:

## Logisitic Regression with Matching Regularizer

In [11]:
matching = pd.read_csv('adult_matches.csv').astype(int)

In [25]:
num_features = df.shape[1]-1
learning_rate = 0.1
lambda_ = 0.1
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.float32, shape=[None, 1])
Mid = tf.constant(matching['male.ids'])
Fid = tf.constant(matching['female.ids'])
W = tf.Variable(tf.zeros([num_features, 1])) # the variables will be all zeros initially
b = tf.Variable(tf.zeros([1]))

In [29]:
Yhat = tf.sigmoid(X @ W + b)
loss = tf.reduce_mean(-(Y*tf.log(Yhat) + (1-Y)*tf.log(1-Yhat)))
Mpred = tf.gather(X,Mid,axis=0) @ W + b
Fpred = tf.gather(X,Fid,axis=0) @ W + b
regularizer = tf.reduce_mean((Mpred - Fpred)**2)
objective = loss + lambda_ * regularizer
# correct_prediction = tf.equal(tf.argmax(Yhat, 1), tf.argmax(Y, 1))           
# accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
optimizer = tf.train.GradientDescentOptimizer(learning_rate) 
update = optimizer.minimize(objective) 

In [30]:
num_epochs  = 500

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        _, loss_val = sess.run([update, loss], feed_dict={X: X_np, 
                                                      Y: Y_np})
        print("Epoch:",epoch,"Cost:",loss_val)        
#     print("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0 Cost: 0.69319
Epoch: 1 Cost: 0.662942
Epoch: 2 Cost: 0.639992
Epoch: 3 Cost: 0.621595
Epoch: 4 Cost: 0.606257
Epoch: 5 Cost: 0.593093
Epoch: 6 Cost: 0.581556
Epoch: 7 Cost: 0.571282
Epoch: 8 Cost: 0.562022
Epoch: 9 Cost: 0.553598
Epoch: 10 Cost: 0.545878
Epoch: 11 Cost: 0.53876
Epoch: 12 Cost: 0.532165
Epoch: 13 Cost: 0.526028
Epoch: 14 Cost: 0.520299
Epoch: 15 Cost: 0.514932
Epoch: 16 Cost: 0.509892
Epoch: 17 Cost: 0.505149
Epoch: 18 Cost: 0.500674
Epoch: 19 Cost: 0.496444
Epoch: 20 Cost: 0.492439
Epoch: 21 Cost: 0.488643
Epoch: 22 Cost: 0.485037
Epoch: 23 Cost: 0.481608
Epoch: 24 Cost: 0.478345
Epoch: 25 Cost: 0.475234
Epoch: 26 Cost: 0.472266
Epoch: 27 Cost: 0.469433
Epoch: 28 Cost: 0.466725
Epoch: 29 Cost: 0.464134
Epoch: 30 Cost: 0.461654
Epoch: 31 Cost: 0.459278
Epoch: 32 Cost: 0.457001
Epoch: 33 Cost: 0.454816
Epoch: 34 Cost: 0.452719
Epoch: 35 Cost: 0.450705
Epoch: 36 Cost: 0.44877
Epoch: 37 Cost: 0.446909
Epoch: 38 Cost: 0.44512
Epoch: 39 Cost: 0.443398
Epoch: 40 Cost

Epoch: 323 Cost: 0.382528
Epoch: 324 Cost: 0.382515
Epoch: 325 Cost: 0.382502
Epoch: 326 Cost: 0.382489
Epoch: 327 Cost: 0.382476
Epoch: 328 Cost: 0.382464
Epoch: 329 Cost: 0.382452
Epoch: 330 Cost: 0.38244
Epoch: 331 Cost: 0.382428
Epoch: 332 Cost: 0.382416
Epoch: 333 Cost: 0.382404
Epoch: 334 Cost: 0.382392
Epoch: 335 Cost: 0.382381
Epoch: 336 Cost: 0.382369
Epoch: 337 Cost: 0.382357
Epoch: 338 Cost: 0.382346
Epoch: 339 Cost: 0.382334
Epoch: 340 Cost: 0.382324
Epoch: 341 Cost: 0.382313
Epoch: 342 Cost: 0.382302
Epoch: 343 Cost: 0.382291
Epoch: 344 Cost: 0.38228
Epoch: 345 Cost: 0.38227
Epoch: 346 Cost: 0.382259
Epoch: 347 Cost: 0.382249
Epoch: 348 Cost: 0.382238
Epoch: 349 Cost: 0.382228
Epoch: 350 Cost: 0.382218
Epoch: 351 Cost: 0.382207
Epoch: 352 Cost: 0.382197
Epoch: 353 Cost: 0.382188
Epoch: 354 Cost: 0.382178
Epoch: 355 Cost: 0.382168
Epoch: 356 Cost: 0.382158
Epoch: 357 Cost: 0.382149
Epoch: 358 Cost: 0.382139
Epoch: 359 Cost: 0.38213
Epoch: 360 Cost: 0.382121
Epoch: 361 Cost:

<tf.Tensor 'add_4:0' shape=(21794, 1) dtype=float32>